In [85]:
import numpy as np
import tensorflowjs as tfjs

from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
import builtins
open = builtins.open


In [86]:
label_path = "../website/prisma/seeding/labels.bin"
digits_path = "../website/prisma/seeding/digits.bin"
model_path = "new_model"

In [87]:
def split_data(x: np.ndarray, y: np.ndarray) -> tuple[tuple[np.ndarray, np.ndarray], tuple[np.ndarray, np.ndarray]]:
    i = int(len(y) * 0.8)
    return (x[:i], y[:i]), (x[i:], y[i:])


# Read labels
labels: np.ndarray
with open(label_path, "rb") as f:
    labels = np.frombuffer(f.read(), dtype=np.uint8)

# Read digits
digits: np.ndarray
size = 28*28
with open(digits_path, "rb") as f:
    digit_buffer = np.frombuffer(f.read(), dtype=np.uint8)

    num_images = len(labels)
    digits = np.empty((num_images, 28, 28))

    for n in range(num_images):
        img = digit_buffer[n*size:(n+1)*size]
        img = img.reshape((28, 28))
        digits[n] = img


# Remove all zeroes
zero_mask = np.where(labels != 0)
labels = labels[zero_mask]
digits = digits[zero_mask]

for i in range(1, 10):
    print(i, len(labels[np.where(labels == i)]))

# Split into train and test
(x_train, y_train), (x_test, y_test) = split_data(digits, labels)

y_train -= 1
y_test -= 1

x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0

x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

1 558
2 560
3 568
4 547
5 539
6 570
7 579
8 555
9 554


In [88]:
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

# non_zero_indices_train = np.where(y_train != 0)[0]
# non_zero_indices_test = np.where(y_test != 0)[0]

# x_train = x_train[non_zero_indices_train]
# y_train = y_train[non_zero_indices_train] - 1

# x_test = x_test[non_zero_indices_test]
# y_test = y_test[non_zero_indices_test] - 1

# x_train = x_train.astype(np.float32) / 255.0
# x_test = x_test.astype(np.float32) / 255.0

# x_train = x_train.reshape(-1, 28, 28, 1)
# x_test = x_test.reshape(-1, 28, 28, 1)


In [89]:
datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
            zoom_range = 0.1, # Randomly zoom image 
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=False,  # randomly flip images
            vertical_flip=False)  # randomly flip images

In [90]:
# datagen.fit(x_train)

In [91]:
y_train = to_categorical(y_train, num_classes=9)
y_test = to_categorical(y_test, num_classes=9)

In [92]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last',
                 input_shape=(28,28,1)))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid' ))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding='same', activation='relu', data_format='channels_last'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), padding='valid', strides=2))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(9, activation='softmax'))

In [93]:
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_28 (Conv2D)          (None, 28, 28, 32)        320       
                                                                 
 batch_normalization_42 (Ba  (None, 28, 28, 32)        128       
 tchNormalization)                                               
                                                                 
 conv2d_29 (Conv2D)          (None, 28, 28, 32)        9248      
                                                                 
 batch_normalization_43 (Ba  (None, 28, 28, 32)        128       
 tchNormalization)                                               
                                                                 
 max_pooling2d_14 (MaxPooli  (None, 14, 14, 32)        0         
 ng2D)                                                           
                                                      

In [94]:
batch_size = 64
epochs = 10

In [95]:
# model.fit(datagen.flow(x_train, y_train, batch_size = batch_size), epochs = epochs, 
#                               validation_data = (x_test, y_test), verbose=1, 
#                               steps_per_epoch=x_train.shape[0] // batch_size)
model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, 
                              validation_data = (x_test, y_test), verbose=1, 
                              steps_per_epoch=x_train.shape[0] // batch_size)

Epoch 1/8


2024-04-25 02:17:05.559047: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


62/62 [==============================] - 3s 31ms/step - loss: 0.6812 - accuracy: 0.8087 - val_loss: 8.4659 - val_accuracy: 0.1133
Epoch 2/8
62/62 [==============================] - 2s 25ms/step - loss: 0.1203 - accuracy: 0.9692 - val_loss: 6.3867 - val_accuracy: 0.1133
Epoch 3/8
62/62 [==============================] - 2s 25ms/step - loss: 0.0640 - accuracy: 0.9813 - val_loss: 10.8255 - val_accuracy: 0.1133
Epoch 4/8
62/62 [==============================] - 1s 23ms/step - loss: 0.0508 - accuracy: 0.9874 - val_loss: 7.1113 - val_accuracy: 0.1133
Epoch 5/8
62/62 [==============================] - 2s 24ms/step - loss: 0.0390 - accuracy: 0.9899 - val_loss: 7.1980 - val_accuracy: 0.2376
Epoch 6/8
62/62 [==============================] - 2s 25ms/step - loss: 0.0419 - accuracy: 0.9902 - val_loss: 3.0916 - val_accuracy: 0.2932
Epoch 7/8
62/62 [==============================] - 1s 24ms/step - loss: 0.0434 - accuracy: 0.9889 - val_loss: 1.2772 - val_accuracy: 0.6412
Epoch 8/8
62/62 [============

In [96]:
tfjs.converters.save_keras_model(model, "./models/new_model")